# Week 1 Exercise — Technical Q&A + Comparison Tool

## What I built

This notebook is a **technical question-and-answer tool** that uses two different models to explain things, then compares their answers.

1. **You ask one technical question** (e.g. “Explain this code” or “What is X?”). You type it in the `question` variable below.

2. **Two models answer the same question:**
   - **GPT** (via OpenAI or OpenRouter) — answers with **streaming**, so the text appears as it’s generated.
   - **Llama 3.2** (via Ollama, running locally) — returns the full answer in one go.

3. **The tool then compares the two answers:** it sends both explanations back to the model and asks: *“Which explanation is clearer, and why?”* You get a short comparison so you can see which one helped more.

## What it does (when you run it)

- **Setup:** Loads your API key from `.env` and connects to OpenAI or OpenRouter (if your key starts with `sk-or-v1-`).
- **Your question:** Edit the `question` cell and re-run from there to ask something new.
- **GPT answer:** Streams the explanation and saves it as `response`.
- **Llama answer:** Gets the explanation from Ollama and saves it as `reply`.
- **Compare:** Calls the API again with both answers and displays which is clearer and why.

## In one sentence

**One technical question → two explanations (GPT + Llama) → one short comparison of which explanation is clearer.** You can reuse this during the course whenever you want two perspectives on a technical topic.

In [40]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [48]:
# constants

MODEL_GPT = 'gpt-4o-mini'
# Use llama3.2:1b if you get "model requires more system memory" (needs ~1.2 GiB). Use llama3.2 if you have 2.5+ GiB free.
# MODEL_LLAMA = 'llama3.2:1b'
MODEL_LLAMA = 'llama3.2'

In [49]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith("sk-or-v1-"):
    openai = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")
    model = "openai/gpt-4o-mini"
else:
    openai = OpenAI()
    model = MODEL_GPT

In [50]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and when you would use it:
@functools.lru_cache(maxsize=128)
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)
"""

In [51]:
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs."
user_prompt = "Please give a detailed explanation to the following question: " + question


messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [52]:
#Get GPT answer (streaming)

stream = openai.chat.completions.create(model=model, messages=messages, stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ""
    update_display(Markdown(response), display_id=display_handle.display_id)

The code you've provided defines a function called `fib` that computes Fibonacci numbers using recursion, while also utilizing a technique called "memoization" through the `lru_cache` decorator from the `functools` module. Let's break this down step by step.

### Explanation of the Code

1. **Decorators in Python**: 
   - The `@functools.lru_cache(maxsize=128)` line is a decorator that applies the Least Recently Used (LRU) cache to the `fib` function. This cache helps store the results of the function calls to avoid redundant computations for the same input.
   - The `maxsize` parameter defines how many results should be cached. In this case, it can cache up to 128 recent calls.

2. **Fibonacci Function**:
   - The function `fib(n)` computes the n-th Fibonacci number. The Fibonacci sequence is defined as:
     - Fib(0) = 0
     - Fib(1) = 1
     - Fib(n) = Fib(n - 1) + Fib(n - 2) for n ≥ 2
   - The first two numbers of the Fibonacci series are 0 and 1, and every subsequent number is the sum of the two preceding numbers.

3. **Base Cases**:
   - The line `if n < 2: return n` handles the base cases of the Fibonacci sequence. If `n` is either 0 or 1, the function will return `n` itself.

4. **Recursive Calls**:
   - If `n` is greater than or equal to 2, the function calls itself recursively to calculate the Fibonacci numbers for `n - 1` and `n - 2`, then adds them together.

### How `lru_cache` Works

- **Efficiency**: Without memoization, calculating `fib(n)` results in an exponential time complexity, specifically O(2^n), due to the repeated calculations of the same Fibonacci values for different calls. Adding the `lru_cache` decorator changes the time complexity to O(n) for this recursive implementation because it stores already computed Fibonacci values. Consequently, when the function is called with the same argument again, it retrieves the value from the cache, rather than calculating it from scratch.
  
- **Storage**: The cache is stored in a fixed-size list, and when it exceeds the specified `maxsize`, the least recently used entries will be discarded.

### When You Would Use This Code

1. **Calculating Fibonacci Numbers**: This code is particularly useful when you need to compute Fibonacci numbers, especially for larger values of `n`, where the recursive approach would normally be inefficient.

2. **Dynamic Programming**: `lru_cache` effectively transforms this recursive approach into a dynamic programming technique by storing intermediate results, thus optimizing the function.

3. **General Memoization Needs**: While this is specifically implemented for Fibonacci numbers, the memoization strategy can be applied to other recursive functions where there are overlapping subproblems.

4. **Performance Considerations**: If you're working in an environment where performance is critical, such as algorithms that are part of larger data processing, machine learning applications, or real-time systems, using `lru_cache` can significantly speed up recursive computations.

### Practical Use Case Example
You might use this function in scenarios where you are creating applications that analyze numerical sequences, such as in financial models that rely on Fibonacci for technical analysis, or in generating sequences for algorithmic solutions where Fibonacci logic is pivotal.

### Final Note
It's worth noting that Python has other ways of calculating Fibonacci numbers in a more efficient manner than recursion (e.g., iterative methods or matrix exponentiation), but using `lru_cache` provides a clear understanding of how memoization works and its benefits in recursive functions.

In [53]:
# Get Ollama answer
ollama_response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = ollama_response["message"]["content"]
display(Markdown(reply))

**Memoized Fibonacci Function**

The given code implements a memoized version of the Fibonacci function, which is a classic example of dynamic programming. Here's a breakdown of what it does and when to use it:

### What is Memoization?

Memoization is an optimization technique that stores the results of expensive function calls so that they can be reused instead of recalculated. This approach avoids redundant calculations and reduces computation time.

### The Code

```python
@functools.lru_cache(maxsize=128)
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)
```

This code defines a function `fib` that calculates the nth Fibonacci number. The `@functools.lru_cache` decorator is used to implement memoization.

Here's how it works:

1. The `lru_cache` decorator caches the results of function calls so they can be reused.
2. When the function `fib` is called with a value of `n`, Python checks if the result for that `n` is already cached.
3. If it is, the cached result is returned immediately.
4. If not, the function calls itself recursively to calculate the result for `n-1` and `n-2`.
5. The results are then combined using addition (`fib(n - 1) + fib(n - 2)`).
6. Once both recursive calls have completed, the final result is returned.

### Why Use Memoization?

Memoization can significantly speed up performance when dealing with:

*   **Computational expensive functions**: When the function has a high number of redundant calculations, memoization can reduce computation time.
*   **Recursive functions**: Memoization is particularly useful for recursive functions that have overlapping subproblems (like the Fibonacci sequence).
*   **Highly dynamic data**: In situations where data is being generated or updated frequently, memoization ensures that you're using up-to-date values.

### Example Use Case

Let's say you need to calculate the 10th Fibonacci number:
```python
fib_n = fib(10)
print(fib_n)  # Output: 55
```

Without memoization, calculating the 10th Fibonacci number could involve redundant calculations (e.g., `fib(8)` is calculated multiple times). With memoization, the result for `n=10` will be cached immediately, avoiding these redundant calculations.

### Potential Drawbacks

While memoization can be a powerful optimization technique, it's essential to consider the following:

*   **Cache size**: The `maxsize` parameter in the `lru_cache` decorator controls how many results are stored. If this number is too small, you might end up with a significant cache size, which could lead to memory issues.
*   **Cache invalidation**: When data changes (e.g., when input values change), memoized results may become outdated.

To mitigate these concerns, consider using techniques like:

*   **Dynamic caching**: Update the cache only when necessary (i.e., when input values change).
*   **Lazy evaluation**: Only calculate results on demand to avoid initial high memory usage.

Compare the two answers

In [54]:
# Compare the two answers
compare_system = "You are a clear, fair reviewer. Compare two explanations and say which is clearer and why, in a short paragraph."
compare_user = "Question: " + question + "\n\nAnswer A (GPT):\n" + response + "\n\nAnswer B (Llama):\n" + reply
compare_messages = [{"role": "system", "content": compare_system}, {"role": "user", "content": compare_user}]

In [55]:
comparison_response = openai.chat.completions.create(model=model, messages=compare_messages)
comparison_text = comparison_response.choices[0].message.content
display(Markdown(comparison_text))


Answer A provides a more comprehensive and clearer explanation of the code than Answer B. It breaks down each component step-by-step, detailing not just the functionality of the `fib` function itself but also how the `lru_cache` decorator works and enhances performance through memoization. Additionally, Answer A elaborates on the potential applications of the code, discussing its relevance in dynamic programming and performance-critical situations, which gives context to its use. On the other hand, Answer B, while still informative, is less detailed and doesn’t explore the implications and broader contexts as thoroughly, making it less effective for someone looking for a full understanding of the code’s purpose and applications.